Nom Etudiant 1: Shiyao CHEN
<br>
Nom Etudiant 2: Ménalie SELLAHENNEDI

# TME 3.1 : Greedy Profile Motif Search 


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm Greedy Profile Motif Search.


1\. Nous allons réutiliser les fonctions du precedent pour générer `t` séquences artificielles de taille `n`, et implanter dans chaque séquence un motif de taille `k` à des positions aléatoires avec `v` substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [34]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

k=5 #taille de motif
v=1 #nb de positions variable dans le motif
t=3 #nb de sequences
n=10 #longuer des sequence
f= 0.5 #frequence de variation du motif.


In [35]:

def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n, upper=True):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront en majuscule, False minuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    for i in range (n):
        s=random.choice(nuc)
        if(upper):
            sequence=sequence+(s.upper())
        else:
            sequence=sequence+(s.lower())
    
    return sequence

def modifierMotif(motif, nbpos,  upper=True):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront majuscule, False minuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)
    #print(motif)
    for i in range(0,nbpos):
        t=random.randint(0,len(motifM)-1)
        while (i>0 and t==prec): #pour verifier qu'on ne s'occupe pas d'une même position 2 fois de suite
            t=random.randint(0,len(motifM)-1)
            
        s=random.choice(nuc)
        #print(s==motifM[t].upper())
        while (s==motifM[t].upper()): #pour verifier qu'on ne substitue pas par la même lettre
            s=random.choice(nuc)
        
        if (upper==True):
            motifM=motifM[:t]+[s.upper()]+motifM[t+1:]
        else:
            motifM=motifM[:t]+[s.lower()]+motifM[t+1:]
        prec=t
    
    #print(motifM)
    return "".join(motifM)

#tester la fonction modifierMotif
#print (modifierMotif("acg", 2))


def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    entrée f : frequence de variation du motif.
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    sequences = []
    for i in range(0,t):
        if(random.uniform(0.0, 1.0)<=f):
            pos=random.randint(0,n)
            sequences.append(insertMotif(generateRandomSequence(n-k,False),modifierMotif(generateRandomSequence(k,False),v),pos))
        else:
            sequences.append(generateRandomSequence(n,False))
    return sequences

adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

['ttacgcacgt', 'tcatagCggt', 'acaagccacg']
['TTACGCACGT', 'TCATAGCGGT', 'ACAAGCCACG']


2\. Faire une fonction pour sélectionner des positions de départ aléatoirement `s = (s1, …,st)`.


In [21]:
#creating vector s
#Get t random positions from 0 to n - k
def generateRandomS(sequences, k):
	"""
	Génère un vecteur de position aléatoires
	entrée sequences: matrice de dimension txn contenant les sequences
	entrée k: taille du motif
	sortie s: vecteur de position aléatoires, une position par séquence
  REMARQUE les positions doivent être inférieur à n-k, ou n est la taille de sequence
	"""
	s = []
	for seq in sequences:
		pos=random.randint(0,n-k)
		s.append(pos)
	return s

s = generateRandomS(adn, k)
print (s)

[5, 3, 0]


3\. Extraire les motifs en utilisant le vecteur `s`, et construire un profile (matrice de fréquence). 


In [22]:
#extract sequences from seqs
def extractSeqs(s, seqs, k):
    """
    Extraire les motifs des séquences à l'aide de positions s
    entrée s: vecteur contenant les positions de départs
    entrée seqs: matrice de dimension txn contenant les séquences
    entrée k: taille du motif
    sortie motifs: liste de motifs de taille k extrait des séquences
    """
    motifs = []  
    ligne=0  
    for seq in seqs:
        pos=s[ligne]
        motif=seq[pos:pos+k]
        motifs.append(motif)
        ligne+=1
    return motifs
        
motifs = extractSeqs(s, adn, k)
print (motifs)


#Construire un profile = matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    q = len(nuc)
    PWM = np.zeros((q, k))
    i=0
    j=0
    for i in range (0,k):
        for j in range (len(motifs)):  # 1er ligne =A  2eme ligne =C   3eme ligne =G   4eme ligne =T
            lettre=motifs[j][i]
            if lettre=="A":
                PWM[0][i]+=1
            if lettre=="C":
                PWM[1][i]+=1
            if lettre=="G":
                PWM[2][i]+=1
            if lettre=="T":
                PWM[3][i]+=1

    return PWM
MF = profile(motifs, k, nuc)
print(MF)

['CTCCG', 'GATCA', 'CCCTC']
[[0. 1. 0. 0. 1.]
 [2. 1. 2. 2. 1.]
 [1. 0. 0. 0. 1.]
 [0. 1. 1. 1. 0.]]


4\. Transformer la matrice de fréquence en PWM. Il faut diviser chaque élément par la somme de ses colonnes.

In [23]:
def somme_colone(MF):
  """
  Calculer la somme de ses colonnes
  entrée MF: matrice de fréquence
  sortie list_som: liste de la somme de chaque colonne 
  """
  list_som=[]
  i=0
  j=0
  for i in range (len(MF[0])):
     list_som.append(0)
     for j in range(len(MF)):
        list_som[i]+=MF[j][i]
  return list_som

def generatePWM(MF):
  """
  Transforme la matrice de fréquence en PWM
  entrée MF: matrice de fréquence
  sortie PWM: matrice de probabilité (poids positions)
  """
  i=0; j=0
  PWM=MF.copy()
  somme=somme_colone(PWM)
  for i in range (0,len(PWM)):
      for j in range(0,len(PWM[i])):
          PWM[i][j]=round(PWM[i][j]/somme[j],2)
  return PWM

PWM = generatePWM(MF)
print(PWM)
#print(MF)

[[0.   0.33 0.   0.   0.33]
 [0.67 0.33 0.67 0.67 0.33]
 [0.33 0.   0.   0.   0.33]
 [0.   0.33 0.33 0.33 0.  ]]


5\. Faire une fonction pour calculer la probabilité d'un motif de taille `k` selon une PWM.


In [24]:
def probability(PWM, motif):
    """
    Calcul la probalité d'un motif selon PWM
    entrée PWM: matrice de probabilité (poids positions)
    entrée motif: motif
    sortie prob: probalité Prob(motif|PWM)
    """
    prob = 1
    i=0
    for i in range (len(motif)):
        if motif[i]=="A":
            p=PWM[0][i]
        if motif[i]=="C":
            p=PWM[1][i]
        if motif[i]=="G":
            p=PWM[2][i]
        if motif[i]=="T":
            p=PWM[3][i]
        #print(p)
        #if (p!=0.0): 
        prob=prob*p
    return prob

prob = probability(PWM, "TTCAT")
#prob = probability(PWM, "AATGA")

print (prob)

0.0


6\. Faire une fonction pour calculer le pMostProbkmer d'une séquence, voir un exemple dans les slides de cours.

In [25]:
def pMostProbkmer(PWM, k, sequence):
    """
    calcul la position du k-mer le plus probable dans la séquence
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: la position la plus probable 
    """
    maxProb = 0
    s = 0 # position du k-mer le plus probable dans la sequence
    i=0
    max_prob=0
    for i in range (len(sequence)-k+1):
            motif=sequence[i:i+k]
            prob=probability(PWM, motif)
            #print(prob)
            if(max_prob<prob):
                max_prob=prob
                #print(max_prob)
                s=i
    #print(max_prob)
    return s
#print(PWM)
ss="tgtaaacttaccagtagatggattgggcttcccggtgaaacgccctatacttggctgagctgttgccaaaaatcttctgg".upper()
#print(ss)
print(pMostProbkmer(PWM, k, ss))
#s = pMostProbkmer(PWM, k, "GCCGGATACT")
#print (s)

27


7\. Faire une fonction pour obtenir les nouvelles positions de départ `s = (s1, …,st)`, c’était à dire les positions qui contiens les k-mer le plus probables.

In [26]:
def getNewS(PWM, k, sequences):
    """
    Trouve les nouvelles positions des k-mer le plus probables
    entrée PWM: matrice de probabilité (poids positions)
    entrée k: taille du motif
    entrée sequence: séquence nucleotidique
    sortie s: vecteur avec les positions les plus problables
    """

    s = []
    for seq in sequences:
        s.append(pMostProbkmer(PWM, k, seq))
    return s

ns = getNewS(PWM, k, adn)
print (ns)

[5, 3, 2]


8\. La condition d’arrêt de l'algorithme est le non changement de la matrice de fréquence d'une itération à l'autre, 
faire une fonction pour comparer deux matrices et détecter le changement. 
Faire aussi une fonction pour obtenir le score d'une matrice de fréquence : la somme de max de chaque colonne

In [27]:
def changeProfile(P1, P2):
    """
    Compare deux matrice
    entrée P1: matrice de fréquence
    entrée P2: matrice de fréquence
    sortie: True si les matrices sont différents, False au contraire
    """
    for i in range(len(P1)):
        for j in range(len(P1[i])):
            if(P1[i][j]!=P2[i][j]):
                return True
    return False

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    sc = 0
    i=0
    j=0
    for i in range(len(MF[0])):
        max_col=0
        for j in range (len(MF)):
            s=MF[j][i]
            if(max_col<s):
                max_col=s
        #print(max_col)
        sc=sc+max_col
    return sc

ns = getNewS(PWM, k, adn)
print (s, ns)

#print(MF)
sc = getScore(MF, k)
print (sc)


[5, 3, 0] [5, 3, 2]
8.0


9\. Implementer l'algorithme ``GreedyProfileMotifSearch`` en utilisant les fonctions precedentes. 
BONUS: Pour ameliorer la performance vous pouvez abandonner les positions qui génèrent des motifs peu complexes.

In [28]:
def removeLowComplexe(motifs,k:int):
    motifsClean = []
    est_supprime=False
    for key in motifs:
        for i in range (len(key)):
            cpt=key.count(key[i])
            if cpt>=k:
                est_supprime=True
            if(i+3< len(key)):
                if key[i]==key[i+2] and key[i+1]==key[i+3]:
                    est_supprime=True     
        if(est_supprime==False):
            motifsClean.append(key)
        est_supprime=False  

    return motifsClean


def GreedyProfileMotifSearch(sequences, k):
    """
    Implémente l'algorithme GreedyProfileMotifSearch 
	  entrée sequences: matrice de dimension txn contenant les séquences 
	  entrée k : taille du motif 
    sortie s : vecteur de positions de départ ayant le meilleur motif 
    sortie bestScore : le score associé à s
    """
    s = generateRandomS(sequences, k)
    #print(s)
    bestScore = 0
    motifs=extractSeqs(s, sequences, k)
    #print(motifs)
    MF=profile(motifs, k, nuc)
    #print(MF)
    PWM=generatePWM(MF)
    score=getScore(MF,k)
    i=0
    
    while (score>bestScore):
        bestScore=score
        for i in range(len(s)):
            p_max=pMostProbkmer(PWM, k, sequences[i])
            s[i]=p_max
            #print(s[i]) 
        motifs=extractSeqs(s, sequences, k)
        #print(motifs)

        MF=profile(motifs, k, nuc)
        #print(MF)
        PWM=generatePWM(MF)
        score=getScore(MF,k)
    return s, bestScore

print (adn)
s, bestScore = GreedyProfileMotifSearch(adn, k) 
print(s,bestScore)
motifs = extractSeqs(s, adn, k)
print (motifs)
        

['CTACTCTCCG', 'GTGGATCAAG', 'CCCTCCGGCA']
[0, 5, 2] 10.0
['CTACT', 'TCAAG', 'CTCCG']


10\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour le trouver. 

Reponse:
Non, je pense on n'a pas trouvé le motif implanté parceque les positions de départ sont générées au hasard, il y a peu de chances que notre chix soit proche d'un motif optimal. 

11\. Vous avez certainement observer que l’algorithme ne produire pas toujours la même sortie et que les résultats dépendent de la sélection aléatoire des positions de départ `s`. Pour augmenter nous chances de retrouvez les bons motifs, nous allons implémenter une version itérative ``GreedyProfileMotifSearchIte`` 
qui a chaque fois sauvegarde `s` et le score du profile associé à `s`, après `I` itération, l’algorithme renvoie le vecteur `s` ayant le plus grand score.

In [29]:
def GreedyProfileMotifSearchIte(sequences, k, It):
    """
    Implémente l'algorithme GreedyProfileMotifSearch iteratif
      entrée séquences: matrice de dimension txn contenant les séquences
      entrée k: taille du motif 
    entrée It: nombre d'iterations
    sortie positions: dictionnaire clé=score, valeur= vecteur s 
    sortie consensus: dictionnaire clé=score, valeur= sequence consensus du motif
    """
   
    pos = {}
    consensus = {}
    
    i=1
    while i<=It:
        s, score=GreedyProfileMotifSearch(sequences, k)
        pos[score]=s
        consensus[score]=extractSeqs(s, sequences, k)
        i+=1
        
    return pos, consensus

print(adn)
s, consensus = GreedyProfileMotifSearchIte(adn, k, 50)

print (s)
print (consensus)

['CTACTCTCCG', 'GTGGATCAAG', 'CCCTCCGGCA']
{11.0: [4, 3, 1], 9.0: [4, 0, 4], 10.0: [4, 2, 1], 8.0: [0, 0, 1], 7.0: [2, 5, 5], 6.0: [2, 0, 0], 12.0: [5, 4, 2]}
{11.0: ['TCTCC', 'GATCA', 'CCTCC'], 9.0: ['TCTCC', 'GTGGA', 'CCGGC'], 10.0: ['TCTCC', 'GGATC', 'CCTCC'], 8.0: ['CTACT', 'GTGGA', 'CCTCC'], 7.0: ['ACTCT', 'TCAAG', 'CGGCA'], 6.0: ['ACTCT', 'GTGGA', 'CCCTC'], 12.0: ['CTCCG', 'ATCAA', 'CTCCG']}


In [30]:
#Affiche les sequence consensus de top motifs
def printTopFMotifsScore(consensus, top):

    motifsSort = sorted(consensus.keys())
    motifsSort.reverse()
    for i in range(top):
        print(i)
        print (consensus[motifsSort[i]])

printTopFMotifsScore(consensus, 2)




0
['CTCCG', 'ATCAA', 'CTCCG']
1
['TCTCC', 'GATCA', 'CCTCC']


12\. Tester algorithme  ``GreedyProfileMotifSearchIte`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats.

In [31]:

def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

In [32]:
k=8
n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_8.fasta"


sequencesChip   = readFasta(genome, n)
for seq in sequencesChip:
    if (len(seq)<k):
        sequencesChip.remove(seq)
        
t = len(sequencesChip)
print (sequencesChip[8], t, k)
revSequences = [reverseComplement(m) for m in sequencesChip]

sequencesChip = sequencesChip + revSequences
print(len(sequencesChip ))

s, consensus = GreedyProfileMotifSearchIte(sequencesChip, k, 20)
printTopFMotifsScore(consensus, 10)


GTGTGAGTCTCGTTGTACGTCCGAGTTCTCGGTTTTTTACGAAACAAATCCCCTTCGGCTGCGCGGTTGCCTCGCGGTCC 399 8
798
0
['TAAACTTA', 'AAAATTTA', 'AAAAAAAA', 'TACTTTTT', 'TACAGCTA', 'CACAGACA', 'ATCAGAAT', 'AATTGACA', 'AACAAATC', 'GTGTGATA', 'ATGTGATT', 'ATGTGATA', 'AAAATATA', 'TACAGCCA', 'TAGTGTGT', 'TAGTAGTA', 'ATCATAAA', 'TAGTGCTG', 'AACATGTA', 'AACATATT', 'TTTATATT', 'AAATGGAA', 'AAATTAAG', 'AACTGGAA', 'TATATAAA', 'ATTATATA', 'TAAAGATA', 'AAAAGATA', 'AAAATATT', 'AAATTAAA', 'ATAATATT', 'AAAAGAAA', 'AAAAGAAA', 'AATAGTTA', 'AAAAGAAA', 'TAATTAAA', 'TAAAGAGA', 'TAAATACA', 'AATTTATA', 'AAAAGTTA', 'AAAATCTA', 'AAAAGACT', 'TTTTGATC', 'AATTTTTC', 'AAGTGTTA', 'ATGATATA', 'AAGTTACA', 'TAATGACT', 'ATCAGCTT', 'AAAAATTA', 'TAGATCTA', 'TTCATGTA', 'CTTTGAAA', 'TTGATGCA', 'AAGATAAA', 'AACAGATA', 'TTCTGAAA', 'TACTCGTA', 'TTGTTATA', 'ATATTATA', 'ATATTATA', 'TTTTTATA', 'ATTATCAA', 'ATTTTACT', 'AAAGCAAA', 'AATAGTTG', 'TTCCCATT', 'CTCTGTTA', 'TATTTCTA', 'AACAATTA', 'AATATTTT', 'AAAACACA', 'ATATGCAA', 'ATGTTATC', 'AAATTGCA', 'TATAT